In [1]:
import numpy as np
import pandas as pd 

import os
import gensim

In [2]:
url = "word2vec_twitter_tokens.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True, unicode_errors='ignore')

In [3]:
#embeddings.most_similar('bitch', topn = 5)
#embeddings.doesnt_match(['apple','banana','flower'])
#embeddings.most_similar(positive = ['king','woman'], negative = ['man'])

In [4]:
colnames=['Tweet', 'Classification']
train_csv = pd.read_csv('C:/Users/U713884/Documents/Nordakademie/WPM Einführung in die KI/OwnPartitioning/hatespeech_train100.csv', names=colnames, header=None, delimiter=';').astype('U')
train_csv.head()
test_csv = pd.read_csv('C:/Users/U713884/Documents/Nordakademie/WPM Einführung in die KI/OwnPartitioning/hatespeech_test.csv', names=colnames, header=None, delimiter=';').astype('U')
test_csv.head()
print (train_csv)
print (test_csv)

                                                 Tweet Classification
0                 bird feather stand peta stand sanity              0
1    wake di morning pocket fill xtra pro slide thr...              1
2           trash snd star scrape ur team snd pro team              0
3                           nt ud pussy yourself pussy              1
4                                   life bitch dog day              1
..                                                 ...            ...
897  iu honorary student id cooler international st...              0
898  girl colored people die holocaust picture holo...              0
899                                        flappy bird              0
900  arent stick garbage recycle im single fuck im ...              1
901                                    post song guala              0

[902 rows x 2 columns]
                                                Tweet Classification
0                    hoe lay puppy eye sayin stay wit              

In [5]:
docs_vectors_train = pd.DataFrame() 
for doc in train_csv['Tweet']:
    temp = pd.DataFrame()  
    for word in doc.split(' '):
        try:
            word_vec = embeddings[word] 
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors_train = docs_vectors_train.append(doc_vector, ignore_index = True) 
docs_vectors_train.shape

(902, 400)

In [6]:
docs_vectors_test = pd.DataFrame()
for doc in test_csv['Tweet']:
    temp = pd.DataFrame()
    for word in doc.split(' '): 
        try:
            word_vec = embeddings[word] 
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors_test = docs_vectors_test.append(doc_vector, ignore_index = True)
docs_vectors_test.shape

(100, 400)

In [7]:
pd.isnull(docs_vectors_train).sum().sum()
pd.isnull(docs_vectors_test).sum().sum()

400

In [8]:
print (docs_vectors_train)

          0         1         2         3         4         5         6    \
0    0.138487 -0.002912 -0.121982 -0.061652 -0.179469 -0.129264 -0.227865   
1    0.128101  0.042784 -0.126150  0.080017  0.071535 -0.243294 -0.106714   
2    0.152572  0.035148 -0.090621  0.069204  0.196073 -0.425802 -0.009700   
3    0.106091  0.217428 -0.271630  0.013729 -0.303250 -0.149806  0.104003   
4    0.258321  0.135977 -0.059272 -0.094361 -0.161200 -0.368803 -0.088933   
..        ...       ...       ...       ...       ...       ...       ...   
897  0.076132  0.044501 -0.096838 -0.030460 -0.034590 -0.408548 -0.101186   
898  0.183686  0.024167  0.054585  0.093391 -0.157928 -0.281188 -0.024889   
899  0.209430  0.200209 -0.155619  0.258292  0.064989  0.052761 -0.230197   
900  0.140382  0.041249 -0.020796 -0.049733  0.157028 -0.083795 -0.014951   
901  0.235855  0.062413 -0.082691 -0.170068  0.151487 -0.259424  0.215690   

          7         8         9    ...       390       391       392  \
0  

In [9]:
docs_vectors_train['Classification'] = train_csv['Classification']
docs_vectors_train = docs_vectors_train.dropna()

docs_vectors_test['Classification'] = test_csv['Classification']
docs_vectors_test = docs_vectors_test.dropna()

In [10]:
from sklearn.ensemble import AdaBoostClassifier

docs_vectors_train.drop('Classification', axis = 1)
docs_vectors_test.drop('Classification', axis = 1)

train_x = docs_vectors_train.drop('Classification', axis = 1)
train_y = docs_vectors_train['Classification']
test_x = docs_vectors_test.drop('Classification', axis = 1)
test_y = docs_vectors_test['Classification']

train_x.shape, train_y.shape, test_x.shape, test_y.shape

((900, 400), (900,), (99, 400), (99,))

In [11]:
model = AdaBoostClassifier(n_estimators=900, learning_rate=0.1, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
from sklearn import metrics

print("Accuracy: %0.3f" % metrics.accuracy_score(test_y, test_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_y, test_pred))

Accuracy: 0.869
confusion matrix:
[[44  4]
 [ 9 42]]
